In [ ]:
# z_mean, z_log_var = encoder(input_img)

# z = z_mean + exp(0.5 * z_log_var) * epsilon

# reconstructed_img = decoder(z)

# model = Model(input_img, reconstructed_img)

In [1]:
import tensorflow.keras
from tensorflow.keras import Input
from tensorflow.keras.layers import Conv2D, Dense, Flatten
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
import numpy as np

img_shape = (28, 28, 1)
batch_size = 16
latent_dim = 2

input_img = Input(shape=img_shape)

x = Conv2D(32, 3, padding='same', activation='relu')(input_img)
x = Conv2D(64, 3, padding='same', activation='relu', strides=(2, 2))(x)
x = Conv2D(64, 3, padding='same', activation='relu')(x)
x = Conv2D(64, 3, padding='same', activation='relu')(x)
shape_before_flattening = K.int_shape(x)

x = Flatten()(x)
x = Dense(32, activation='relu')(x)

z_mean = Dense(latent_dim)(x)
z_log_var = Dense(latent_dim)(x)

In [2]:
from tensorflow.keras.layers import Lambda

def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
                              mean=0., stddev=1.)
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling)([z_mean, z_log_var])

In [ ]:
from tensorflow.keras.layers import Reshape, Conv2DTranspose

decoder_input = Input(K.int_shape(z)[1:])

z = Dense(np.prod(shape_before_flattening[1:]),
          activation='relu')(decoder_input) # 입력을 업샘플링

x = Reshape(shape_before_flattening[1:])(x)
x = Conv2DTranspose(32, 3, 
                    padding='same',
                    activation='relu',
                    strides=(2, 2))(x)
x = Conv2D(1, 3, 
           padding='same',
           activation='sigmoid')(x)

decoder = Model(decoder_input, x)